<font color='maroon' size=5A face="black body"> Get data </font>



In [13]:
%store -r dbcopy 
%store -r df_task2
%store -r user_behaviour
%store -r network_per_user_df

db = dbcopy

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cluster import KMeans


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pickle

In [25]:
min_max_scaler = preprocessing.MinMaxScaler()
df_values = df_task2.values
scalled_values = min_max_scaler.fit_transform(df_values)
df_normalized = pd.DataFrame(data=scalled_values, columns=df_task2.columns)
kmeans = KMeans(n_clusters=3).fit(df_normalized)
cluster = kmeans.predict(df_normalized)
engagement_df = df_task2.copy()
engagement_df['cluster-engagement']  = cluster
## Engagement Score
lowest_engagement = engagement_df.groupby('cluster-engagement').get_group(0).mean()
lowest_experiance = experiance_df.groupby('cluster-experiance').get_group(0).mean()
lowest_engagement

Bearer Id             1.042692e+00
Dur. (ms)             9.331032e+04
Total                 4.992107e+08
cluster-engagement    0.000000e+00
dtype: float64

In [11]:
def get_engagement_score(df, lowest):
    x = float(lowest['Bearer Id'])
    y = float(lowest['Dur. (ms)'])
    z = float(lowest['Total'])
    new_df = df.copy()
    new_df['engagement score'] = ((df['Bearer Id'] - x)**2 + (df['Dur. (ms)'] - y)**2 + (df['Total'] - z)**2)**0.5
    return new_df
engagement_scored_df = get_engagement_score(engagement_df, lowest_engagement)
engagement_scored_df.head()

,Bearer Id,Dur. (ms),Total,cluster-engagement,engagement score
MSISDN/Number,,,,,
3.360100e+10,1,116720.0,8.786906e+08,0,3.794799e+08
3.360100e+10,1,181230.0,1.568596e+08,0,3.423511e+08
3.360100e+10,1,134969.0,5.959665e+08,0,9.675577e+07
3.360101e+10,1,49878.0,4.223207e+08,0,7.689003e+07
3.360101e+10,2,37104.0,1.457411e+09,2,9.582002e+08


In [26]:
experiance_df = network_per_user_df.copy()
experiance_df['cluster-experiance']  = cluster
experiance_df = experiance_df.set_index('MSISDN/Number')

def get_experiance_score(df, low):
    x = float(low['Total RTT'])
    y = float(low['Total TCP Retrans'])
    z = float(low['Total Throughput'])
    new_df = df.copy()
    new_df['experience score'] = ((df['Total RTT'] - x)**2 + (df['Total TCP Retrans'] - y)**2 \
                              + (df['Total Throughput'] - z)**2 )**0.5
    return new_df
experiance_scored_df = get_experiance_score(experiance_df, lowest_experiance)
experiance_scored_df.head()

,Handset Type,Total TCP Retrans,Total Throughput,Total RTT,cluster-experiance,experience score
MSISDN/Number,,,,,,
3.360100e+10,Huawei P20 Lite Huawei Nova 3E,2.156957e+07,74.0,46.000000,0,2.644706e+07
3.360100e+10,Apple iPhone 7 (A1778),2.156957e+07,96.0,31.000000,0,2.644706e+07
3.360100e+10,undefined,2.156957e+07,96.0,127.458589,0,2.644706e+07
3.360101e+10,Apple iPhone 5S (A1457),7.607247e+05,408.0,84.000000,0,4.725587e+07
3.360101e+10,Apple iPhone Se (A1723),3.094040e+07,80790.0,119.000000,2,1.707616e+07


In [27]:
satisfaction_df = pd.merge(engagement_scored_df["engagement score"], experiance_scored_df['experience score'], on='MSISDN/Number')
satisfaction_df['satisfaction score'] = (satisfaction_df['engagement score'] + satisfaction_df['experience score']) / 2

In [28]:
satisfaction_df.sort_values(by='satisfaction score', ascending=False).head(5)

,engagement score,experience score,satisfaction score
MSISDN/Number,,,
3.366773e+10,1.706277e+09,8.306193e+09,5.006235e+09
3.365908e+10,1.153621e+09,7.893844e+09,4.523732e+09
3.366098e+10,1.672949e+09,7.326882e+09,4.499916e+09
3.376264e+10,2.763730e+09,5.831973e+09,4.297851e+09
3.361489e+10,8.347016e+09,8.640457e+07,4.216710e+09


In [29]:
regretion_df = pd.merge(engagement_df[['Bearer Id', 'Dur. (ms)', 'Total']],\
                        experiance_df[['Total RTT','Total TCP Retrans', 'Total Throughput']],\
                       on='MSISDN/Number')
regretion_df = pd.merge(regretion_df,satisfaction_df['satisfaction score'], on='MSISDN/Number' )
regretion_df.head()

,Bearer Id,Dur. (ms),Total,Total RTT,Total TCP Retrans,Total Throughput,satisfaction score
MSISDN/Number,,,,,,,
3.360100e+10,1,116720.0,8.786906e+08,46.000000,2.156957e+07,74.0,2.029635e+08
3.360100e+10,1,181230.0,1.568596e+08,31.000000,2.156957e+07,96.0,1.843991e+08
3.360100e+10,1,134969.0,5.959665e+08,127.458589,2.156957e+07,96.0,6.160141e+07
3.360101e+10,1,49878.0,4.223207e+08,84.000000,7.607247e+05,408.0,6.207295e+07
3.360101e+10,2,37104.0,1.457411e+09,119.000000,3.094040e+07,80790.0,4.876382e+08
